In [ ]:
# Import required libraries
from netgen.occ import *
from ngsolve import *
import random
from ngsolve.webgui import *

Parameters to play around with

In [ ]:
a = 2 # domain size
maxh = 0.04 # mesh size
tau = 0.001 # timestep size
gamma = 0.02 ** 2 # length of transition region
D = 1 # diffusion coefficient
phi = 0.5 # average initial phase in [0,1]
explicit = True # Explicit/implicit time stepping

random.seed(1) # Initialize random number generator -> Gives same random numbers with this line.

Create geometry. Use periodic identification for left and right edge and top and bottom one.

In [ ]:
r = WorkPlane().RectangleC(a,a).Face()
r.edges.Min(X).Identify(r.edges.Max(X),
                        "perx", IdentificationType.PERIODIC)
r.edges.Min(Y).Identify(r.edges.Max(Y),
                        "pery", IdentificationType.PERIODIC)
geo = OCCGeometry(r, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=maxh))
Draw(mesh)

Create finite element space and solution functions

In [ ]:
V = Periodic(H1(mesh, order=3))
fes = V*V
(u,p), (v,q) = fes.TnT()

gf = GridFunction(fes)
gfold = GridFunction(fes)

gfu, gfp = gf.components
gfuold, gfpold = gfold.components

Random initial data in `[-0.2, 0.2]`

In [ ]:
for i in range(mesh.nv):
    if(fes.CouplingType(i) == COUPLING_TYPE.WIREBASKET_DOF):
        gfu.vec[i] = 2*phi-1  + (0.4*random.random()-0.2)
Draw(gfu)

We derive a numerical scheme for the equation

$$
u_t = D \Delta (u^3 - u - \gamma \Delta u)
$$

where $u \in [-1,1]$ is the concentation of the fluid. $D$ is a diffusion coefficient with unit $m^2/s$ and 
$\sqrt{\gamma}$ gives the length of the transition regions between the domains.

$\mu = u^3 - u - \gamma \Delta u$ is the chemical potential.

Now define $p := -\Delta u$ and descretize in time with $u_t \approx \tau^{-1} (u - u_o)$ then

$$
\tau^{-1}(u-u_o) = D (3 u^2 - 1) \Delta u + D \gamma \Delta p
$$

Now multiply by $\tau$ bring everything to the same side, integrate and test with testfunctions and do partial integration on all $\Delta$ parts:

$$
\int_\Omega (u-u_o) v + \tau D (3 \tilde u^2 - 1) \nabla \tilde u \nabla v + \tau D \gamma \nabla p \nabla v = 0
$$

The second equation becomes:

$$
\int_\Omega \nabla u \nabla q - p q \, dx = 0
$$

With $\tilde u = u_o$ if explicit, $ = u$ else. 

In [ ]:
unl = gfuold if explicit else u

a = BilinearForm(fes)
a += (u-gfuold)*v * dx
a += tau * D * gamma * grad(p) * grad(v) * dx
a += tau * D * (3*unl**2-1) * grad(unl) * grad(v) * dx
a += (grad(u) * grad(q) - p*q) * dx

For the explicit system we can set up a solve a priori:

In [ ]:
if explicit:
    with TaskManager():
        a.Assemble()
        inv = a.mat.Inverse(fes.FreeDofs(), inverse="pardiso")
        res = gf.vec.CreateVector()    

Do timestepping:

In [ ]:
scene = Draw(gfu, autoscale=False, min=-1, max=1)
with TaskManager():
    for i in range(1000):
        gfold.vec.data = gf.vec
        if explicit:
            a.Apply(gf.vec, res)
            gf.vec.data -= inv * res
        else:
            solvers.Newton(a, gf, printing=True)
        scene.Redraw()
        print("t = ", i*tau, "                ", end="\r")